In [1]:
# import dependencies
import datetime as dt
import pandas as pd
import numpy as np
import requests, io, zipfile, os, time
import dask.dataframe as dd

In [2]:
# read in full csv and scale for analysis

# for curiosity, time taken to read data
## dask documentation from https://www.geeksforgeeks.org/working-with-large-csv-files-in-python/
s_time = time.time()

# Read CSV files from List
ddf = dd.read_csv("full_raw_data.csv",
            low_memory=False,
            dtype={
                    'end_station_id': 'object',
                    'start_station_id': 'object'
                  },
                 )

# end of data read, print time report
e_time = time.time()
print("Processed in ", ((e_time-s_time)/60), "minutes")

ddf.dtypes

Processed in  0.0005865136782328288 minutes


Unnamed: 0              int64
ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object

In [3]:
# dask compute the DataFrame values
##  to create the pandas DataFrame

s_time = time.time()
df = ddf.compute()
e_time = time.time()
print("Processed in ", ((e_time-s_time)/60), "minutes")

Processed in  4.628413697083791 minutes


In [4]:
# # create pandas DataFrame

# s_time = time.time()
# df = pd.DataFrame(ddf)
# e_time = time.time()
# print("Processed in ", ((e_time-s_time)/60), "minutes")

In [5]:
print(f'df contains {df.shape[0]} rows and {df.shape[1]} columns.')
print('*' * 50)
df.dtypes

df contains 57256026 rows and 14 columns.
**************************************************


Unnamed: 0              int64
ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object

In [6]:
# save a copy to test changes
df_copy = df

In [7]:
# for test changes only
# df = df_copy

In [9]:
# set up date formats and rename columns

# check time run
s_time = time.time()

##  start date/time
df["ride_start"] = df["started_at"].astype('datetime64[ns]')

df['ride_year'] = df['ride_start'].dt.year
df['ride_month'] = df['ride_start'].dt.month_name()
df['ride_weekday'] = df['ride_start'].dt.day_name()
df['ride_hour'] = df['ride_start'].dt.hour


##  end date/time
df["ride_end"] = df["ended_at"].astype('datetime64[ns]')

## add column for ride time
# df['ride_minutes'] = df["ended_at"] - df["started_at"]

# rename columns
df = df.rename(columns={
                        'start_station_name' : 'start_station',
                        'end_station_name': 'end_station',
                        'rideable_type': 'ride_type',
                        'member_casual' : 'member_type'
                        })
# print time run
e_time = time.time()
print("Processed in ", ((e_time-s_time)/60), "minutes")

Processed in  7.9642427603403725 minutes


In [10]:
df

,Unnamed: 0,ride_id,ride_type,started_at,ended_at,start_station,start_station_id,end_station,end_station_id,start_lat,start_lng,end_lat,end_lng,member_type,ride_start,ride_year,ride_month,ride_weekday,ride_hour,ride_end
0,0,55262E4365A955A2,classic_bike,2022-01-18 08:23:52,2022-01-18 08:28:18,Boerum Pl\t& Pacific St,4488.09,Clinton St & Joralemon St,4605.04,40.688489,-73.991160,40.692395,-73.993379,member,2022-01-18 08:23:52,2022,January,Tuesday,8,2022-01-18 08:28:18
1,1,D272F1B15D841EC0,classic_bike,2022-01-21 09:03:22,2022-01-21 09:05:44,E 12 St & Ave C,5616.08,E 10 St & Avenue A,5659.05,40.727243,-73.976831,40.727408,-73.981420,member,2022-01-21 09:03:22,2022,January,Friday,9,2022-01-21 09:05:44
2,2,D1FCEF55EB4A807F,classic_bike,2022-01-22 14:28:32,2022-01-22 14:53:18,W 21 St & 6 Ave,6140.05,W 44 St & 11 Ave,6756.05,40.741740,-73.994156,40.762009,-73.996975,member,2022-01-22 14:28:32,2022,January,Saturday,14,2022-01-22 14:53:18
3,3,E9CBDC6A0162C068,electric_bike,2022-01-19 14:49:47,2022-01-19 14:54:02,38 St & 30 Ave,6850.01,Crescent St & 30 Ave,6958.06,40.764175,-73.915840,40.768692,-73.924957,member,2022-01-19 14:49:47,2022,January,Wednesday,14,2022-01-19 14:54:02
4,4,2177A5B57326CE9B,electric_bike,2022-01-16 14:36:06,2022-01-16 14:44:06,Pacific St & Nevins St,4362.04,Clinton St & Tillary St,4748.07,40.685376,-73.983021,40.696233,-73.991421,member,2022-01-16 14:36:06,2022,January,Sunday,14,2022-01-16 14:44:06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172023,1748282,714D3CCFB28DC79C,classic_bike,2021-12-06 15:33:18,2021-12-06 15:42:18,E 31 St & 3 Ave,6239.08,W 13 St & 5 Ave,5947.04,40.743943,-73.979661,40.735445,-73.994310,member,2021-12-06 15:33:18,2021,December,Monday,15,2021-12-06 15:42:18
172024,1748283,1A2F3968E4A735FB,docked_bike,2021-12-11 14:24:00,2021-12-12 13:58:42,MacDougal St & Washington Sq,5797.01,E 2 St & 2 Ave,5593.02,40.732264,-73.998522,40.725029,-73.990697,casual,2021-12-11 14:24:00,2021,December,Saturday,14,2021-12-12 13:58:42
172025,1748284,1BD68D08120346F9,classic_bike,2021-12-05 14:15:22,2021-12-05 14:23:28,Schermerhorn St & Court St,4565.04,Warren St & Court St,4413.08,40.691029,-73.991834,40.686371,-73.993833,member,2021-12-05 14:15:22,2021,December,Sunday,14,2021-12-05 14:23:28
172026,1748285,F7D2A9EAE112CF48,classic_bike,2021-12-31 11:22:45,2021-12-31 11:57:19,Emerson Pl & Myrtle Ave,4683.02,4 Ave & 9 St,3955.05,40.693631,-73.962236,40.670513,-73.988766,member,2021-12-31 11:22:45,2021,December,Friday,11,2021-12-31 11:57:19


In [ ]:
# df2 = df[(df.ride_type != 'docked_bike') & (df.start_station.notnull()) & (df.end_station.notnull())]
# df2

In [ ]:
# df = df2

In [11]:
df.dtypes

Unnamed: 0                   int64
ride_id                     object
ride_type                   object
started_at                  object
ended_at                    object
start_station               object
start_station_id            object
end_station                 object
end_station_id              object
start_lat                  float64
start_lng                  float64
end_lat                    float64
end_lng                    float64
member_type                 object
ride_start          datetime64[ns]
ride_year                    int64
ride_month                  object
ride_weekday                object
ride_hour                    int64
ride_end            datetime64[ns]
dtype: object

In [13]:
# select columns for final csv file
df_cleaned = df


df_cleaned = pd.DataFrame(df, columns={
                                    'start_station',    
                                    'end_station',
                                    'ride_year',
                                    'ride_month',
                                    'ride_weekday',
                                    'ride_hour',
                                    'ride_start',
                                    'ride_end',
                                    'ride_type',
                                    'member_type'    
                                    })
# change order
df_cleaned = df_cleaned.reindex(columns=[
                                    'start_station',    
                                    'end_station',
                                    'ride_year',
                                    'ride_month',
                                    'ride_weekday',
                                    'ride_hour',
                                    'ride_start',
                                    'ride_end',
                                    'ride_type',
                                    'member_type'])

In [14]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57256026 entries, 0 to 172027
Data columns (total 10 columns):
 #   Column         Dtype         
---  ------         -----         
 0   start_station  object        
 1   end_station    object        
 2   ride_year      int64         
 3   ride_month     object        
 4   ride_weekday   object        
 5   ride_hour      int64         
 6   ride_start     datetime64[ns]
 7   ride_end       datetime64[ns]
 8   ride_type      object        
 9   member_type    object        
dtypes: datetime64[ns](2), int64(2), object(6)
memory usage: 4.7+ GB


In [15]:
df_cleaned

,start_station,end_station,ride_year,ride_month,ride_weekday,ride_hour,ride_start,ride_end,ride_type,member_type
0,Boerum Pl\t& Pacific St,Clinton St & Joralemon St,2022,January,Tuesday,8,2022-01-18 08:23:52,2022-01-18 08:28:18,classic_bike,member
1,E 12 St & Ave C,E 10 St & Avenue A,2022,January,Friday,9,2022-01-21 09:03:22,2022-01-21 09:05:44,classic_bike,member
2,W 21 St & 6 Ave,W 44 St & 11 Ave,2022,January,Saturday,14,2022-01-22 14:28:32,2022-01-22 14:53:18,classic_bike,member
3,38 St & 30 Ave,Crescent St & 30 Ave,2022,January,Wednesday,14,2022-01-19 14:49:47,2022-01-19 14:54:02,electric_bike,member
4,Pacific St & Nevins St,Clinton St & Tillary St,2022,January,Sunday,14,2022-01-16 14:36:06,2022-01-16 14:44:06,electric_bike,member
...,...,...,...,...,...,...,...,...,...,...
172023,E 31 St & 3 Ave,W 13 St & 5 Ave,2021,December,Monday,15,2021-12-06 15:33:18,2021-12-06 15:42:18,classic_bike,member
172024,MacDougal St & Washington Sq,E 2 St & 2 Ave,2021,December,Saturday,14,2021-12-11 14:24:00,2021-12-12 13:58:42,docked_bike,casual
172025,Schermerhorn St & Court St,Warren St & Court St,2021,December,Sunday,14,2021-12-05 14:15:22,2021-12-05 14:23:28,classic_bike,member
172026,Emerson Pl & Myrtle Ave,4 Ave & 9 St,2021,December,Friday,11,2021-12-31 11:22:45,2021-12-31 11:57:19,classic_bike,member


In [17]:
# send  cleaned df to csv file

# check time run
s_time = time.time()

path = f'clean_data.csv'
df_cleaned.to_csv(path)

# print time run
e_time = time.time()
print("Processed in ", ((e_time-s_time)/60), "minutes")

Processed in  13.335593835512798 minutes
